## Procesamiento de Texto 📚

El objetivo principal de esta clase es introducirlos a la clasificación de texto en NLP. 
Para esto, implementaremos varios modelos de clasificación destinados a **predecir la categoría de noticias de la radio biobio**.

Los modelos y métodos que usaremos serán los siguientes: 

### 1. **Preprocesamiento**:
El preprocesamiento es el conjunto de técnicas aplicadas a los datos antes de ser utilizados por un modelo de aprendizaje automático. Las técnicas que usaremos son:

- **Tokenización**: Es el proceso de dividir un texto en unidades más pequeñas llamadas *tokens*. Los tokens pueden ser palabras, frases o incluso caracteres individuales, según el nivel de granularidad que se necesite. Por ejemplo, la frase "Hola mundo" se tokeniza como ["Hola", "mundo"].

- **Stemming**: Es una técnica que reduce las palabras a su raíz o forma base. El objetivo es agrupar diferentes formas de una palabra, de modo que puedan tratarse como la misma entidad. Por ejemplo, "corriendo", "corre" y "correr" se reducirían a la raíz "corr".

- **Lematización**: Similar al stemming, pero en lugar de reducir las palabras a su raíz, la lematización convierte las palabras a su forma base o lema según el contexto. Por ejemplo, "correrá" se convierte en "correr". La lematización tiene en cuenta el significado gramatical, lo que la hace más precisa que el stemming.

- **Eliminación de Stop Words**: Las *stop words* son palabras comunes que no aportan mucho significado a la tarea de análisis, como "el", "la", "en", "de". Eliminar estas palabras puede ayudar a reducir el ruido en los datos y mejorar la eficiencia del modelo.

### 2. **Bag of Words**:
El modelo de **Bag of Words (BoW)** es una técnica simple de representación de texto donde un documento o conjunto de palabras se convierte en un conjunto de palabras únicas y su frecuencia dentro de ese documento. Se ignora la gramática y el orden de las palabras, y solo se cuenta cuántas veces aparece cada palabra en el documento. Es un enfoque común para convertir texto en datos que pueden ser usados por modelos de aprendizaje automático. El resultado suele ser una matriz en la que cada fila representa un documento y cada columna representa una palabra única.

### 3. **Clasificador de Bayes**:
El **Clasificador de Bayes** es un algoritmo de clasificación basado en el teorema de Bayes, que calcula la probabilidad de que una instancia pertenezca a una clase dada, en función de sus características. El modelo de Bayes más conocido es el **Naive Bayes**, que asume que las características son independientes entre sí. A pesar de esta suposición ingenua, Naive Bayes es muy eficaz para tareas de clasificación de texto como el filtrado de spam y la categorización de documentos.

### 4. **Logistic Regression (Regresión Logística)**:
La **Regresión Logística** es un modelo de clasificación supervisado que se utiliza para predecir la probabilidad de que una instancia pertenezca a una de dos clases posibles. Aunque se llama "regresión", se utiliza principalmente para problemas de clasificación binaria. La regresión logística estima la probabilidad de que una instancia pertenezca a una clase basándose en una función sigmoide que produce un valor entre 0 y 1. Si el valor es mayor que un umbral (generalmente 0.5), se clasifica la instancia en una clase; de lo contrario, en la otra.

In [ ]:
import pandas as pd    
import spacy
import nltk 

from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

from spacy.lang.es.stop_words import STOP_WORDS
nlp = spacy.load("es_core_news_sm", disable=['ner', 'parser', 'tagger'])

In [ ]:
#! python -m spacy download es_core_news_sm


Entonces, 

### ¿Qué es la clasificación de texto?

La clasificación de texto consiste en tomar distintos textos y asignarles alguna clase. Dichas clases varían según la task que queramos resolver. Por ejemplo:

    - Detectar emails SPAM -> SPAM, NO SPAM
    - Reviews de Peliculas -> Buena, Mas o menos, Mala, Malísima, Brutalmente mala.
    - Análisis de sentimientos de tweets: Felicidad, Tristeza, Enojo, Ira,...
    - Detectar Fake News -> Es, No es
    - Lenguaje del texto -> Español, Inglés, Chino,...
    - Categoría de una noticia -> Nacional, Internacional, Economía, Sociedad, Opinión...
    - Autor de un texto -> Cada autor es una clase distinta.
Se define formalmente como:

- Input: 

    - Un documento $d$
    - Un conjunto fijo de clases $c_1, c_2, ..., c_j$

- Output: 
    
    - Una clase $c \in C$ para el documento 
    
 
Hay dos clases de métodos para resolver estos problemas: 

1. **Hand-coded Rules 🤙**: 

    Establecemos a mano las reglas que permiten detectar las clases.
    

2. **Supervised Machine Learning 💻**:
   
    Entrenamos clasificadores a partir de muchos ejemplos de documentos etiquetados a mano. 
    

----------------------------------------


## ¿Qué haremos a continuación?


Clasificaremos las noticias de la radio biobio en 20 categorías o tópicos:

```python
[
    'america-latina', 'eeuu', 'europa', 'chile', 'region-metropolitana',
    'region-del-bio-bio', 'negocios-y-empresas', 'region-de-los-lagos',
    'actualidad-economica', 'region-de-valparaiso', 'region-de-la-araucania',
    'curiosidades', 'asia', 'region-de-los-rios', 'entrevistas', 'debates',
    'mediooriente', 'viral', 'animales', 'tu-bolsillo'
]
```

Los pasos a seguir serán: 

1. Primero que nada, descargaremos los datos con los que trabajaremos.

2. Luego, crearemos el sistema mas básico. Este consiste en transformar nuestro texto a `Bag of Words (BoW)` y luego, usar esos vectores para entrenar un clasificador. Este sistema nos puede entregar un muy buen baseline para comenzar a mejorar.

3. Evaluaremos nuestro clasificador según las métricas.

4. A continuación, veremos como mejorar aun mas nuestros resultados. Para esto agregaremos muchas mas técnicas vistas en cátedra, tales como el preprocesamiento de texto y probar con clasificadores aún mas sofisticados.



### Cargar los datasets 


Los datos que usaremos son 5000 documentos con noticias dividas en 20 categorías. Las noticias fueron obtenidas desde la página de la radio biobio.
Cada categoría contiene 250 documentos (noticias). 

Los cargaremos directamente desde un archivo utilizando la librería `pandas` 🐼: 


In [ ]:
dataset = pd.read_json('datos/biobio_clean.json')
dataset_r = dataset.copy(deep=True) # respaldo

In [ ]:
dataset

,author,author_link,title,link,category,subcategory,content,tags,embedded_links,publication_datetime
0,Yerko Roa,/lista/autores/yroa,Colapsa otro segmento de casa que se derrumbó ...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,Noticia en Desarrollo Estamos recopilando m...,[],[],1565778000000
1,Valentina González,/lista/autores/vgonzalez,Policía busca a mujer acusada de matar a su pa...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Detectives de la Policía de Investigaciones ...,"[#parricidio, #PDI, #Pudahuel, #Región Metropo...",[https://media.biobiochile.cl/wp-content/uploa...,1565771820000
2,Felipe Delgado,/lista/autores/fdelgado,Dos detenidos en Liceo de Aplicación: protagon...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Dos detenidos fue el saldo de una serie de i...,"[#Incendio, #Liceo de Aplicación, #Región Metr...",[],1565772480000
3,Matías Vega,/lista/autores/mvega,Apoyo transversal: Senado aprueba en general p...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,La sala del Senado aprobó en general el proy...,"[#Inmigración, #Inmigrantes, #Ley, #Migración,...",[https://media.biobiochile.cl/wp-content/uploa...,1565772720000
4,Valentina González,/lista/autores/vgonzalez,Evacuación espontánea en Instituto Nacional po...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,La mañana de este miércoles se produjo una e...,"[#Carabineros, #FFEE, #Gases Lacrimógenos, #In...",[],1565772960000
...,...,...,...,...,...,...,...,...,...,...
26408,Manuel Stuardo,/lista/autores/mstuardo,Naciones Unidas abre proceso de postulaciones ...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,Las Naciones Unidas abrió un proceso de post...,"[#cambio climático, #COP25, #Naciones Unidas, ...",[],1565764200000
26409,Felipe Delgado,/lista/autores/fdelgado,Fernando Astengo chocó en estado de ebriedad e...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,El exfutbolista Fernando Astengo protagonizó...,"[#Accidente, #Fernando Astengo, #Peñalolén, #R...",[https://media.biobiochile.cl/wp-content/uploa...,1565767440000
26410,Felipe Delgado,/lista/autores/fdelgado,Detuvieron a hombre que arrojó combustible a u...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Personal de Carabineros detuvo a un hombre q...,"[#Indigente, #Parque Forestal, #Región Metropo...",[],1565769300000
26411,Nicolás Parra,/lista/autores/nparra,Revelan identidad de 2 de 6 víctimas fatales e...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,"El intendente de Valparaíso, Jorge Martínez,...","[#derrumbe en valparaíso, #Región de Valparaís...",[],1565771100000


In [ ]:
# El número de noticias por clase lo pueden cambiar despues modificando la constante NUM_SAMPLES.
# noten que el número de noticias en el dataset original por categoría está desbalanceada.
# sample intentará sacar la mayor cantidad de ejemplos y retornará siempre, incluso si devuelve 
# menos de los que le pidieron.

NUM_SAMPLES = 250

categorias = dataset['subcategory'].unique()
categorias

array(['region-de-valparaiso', 'region-metropolitana', 'chile',
       'region-de-los-lagos', 'region-del-maule', 'entrevistas', 'tu-voz',
       'america-latina', 'negocios-y-empresas', 'europa',
       'actualidad-economica', 'eeuu', 'africa', 'curiosidades', 'mundo',
       'asia', 'tu-bolsillo', 'mediooriente', 'debate', 'animales',
       'misterios', 'viral', 'oceania', 'consejos', 'salud',
       'region-del-bio-bio', 'region-de-la-araucania',
       'region-de-los-rios', 'region-de-magallanes',
       'region-de-antofagasta', 'region-de-ohiggins', 'region-de-atacama',
       'region-de-tarapaca', 'region-de-aysen', 'region-de-coquimbo',
       'region-de-arica-y-parinacota', 'region-de-nuble', 'videos',
       'educacion-group-nacional', 'viajes', 'fotos-ciudadanas',
       'cronicas', 'test'], dtype=object)

In [ ]:
categorias_seleccionadas = [
    'america-latina', 'eeuu', 'europa', 'chile', 'region-metropolitana',
    'region-del-bio-bio', 'negocios-y-empresas', 'region-de-los-lagos',
    'actualidad-economica', 'region-de-valparaiso', 'region-de-la-araucania',
    'curiosidades', 'asia', 'region-de-los-rios', 'entrevistas', 'debates',
    'mediooriente', 'viral', 'animales', 'tu-bolsillo'
]

In [ ]:
# Filtrar solo las categorías seleccionadas
dataset_filtrado = dataset[dataset['subcategory'].isin(categorias_seleccionadas)]
dataset_filtrado


,author,author_link,title,link,category,subcategory,content,tags,embedded_links,publication_datetime
0,Yerko Roa,/lista/autores/yroa,Colapsa otro segmento de casa que se derrumbó ...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,Noticia en Desarrollo Estamos recopilando m...,[],[],1565778000000
1,Valentina González,/lista/autores/vgonzalez,Policía busca a mujer acusada de matar a su pa...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Detectives de la Policía de Investigaciones ...,"[#parricidio, #PDI, #Pudahuel, #Región Metropo...",[https://media.biobiochile.cl/wp-content/uploa...,1565771820000
2,Felipe Delgado,/lista/autores/fdelgado,Dos detenidos en Liceo de Aplicación: protagon...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Dos detenidos fue el saldo de una serie de i...,"[#Incendio, #Liceo de Aplicación, #Región Metr...",[],1565772480000
3,Matías Vega,/lista/autores/mvega,Apoyo transversal: Senado aprueba en general p...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,La sala del Senado aprobó en general el proy...,"[#Inmigración, #Inmigrantes, #Ley, #Migración,...",[https://media.biobiochile.cl/wp-content/uploa...,1565772720000
4,Valentina González,/lista/autores/vgonzalez,Evacuación espontánea en Instituto Nacional po...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,La mañana de este miércoles se produjo una e...,"[#Carabineros, #FFEE, #Gases Lacrimógenos, #In...",[],1565772960000
...,...,...,...,...,...,...,...,...,...,...
26408,Manuel Stuardo,/lista/autores/mstuardo,Naciones Unidas abre proceso de postulaciones ...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,Las Naciones Unidas abrió un proceso de post...,"[#cambio climático, #COP25, #Naciones Unidas, ...",[],1565764200000
26409,Felipe Delgado,/lista/autores/fdelgado,Fernando Astengo chocó en estado de ebriedad e...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,El exfutbolista Fernando Astengo protagonizó...,"[#Accidente, #Fernando Astengo, #Peñalolén, #R...",[https://media.biobiochile.cl/wp-content/uploa...,1565767440000
26410,Felipe Delgado,/lista/autores/fdelgado,Detuvieron a hombre que arrojó combustible a u...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Personal de Carabineros detuvo a un hombre q...,"[#Indigente, #Parque Forestal, #Región Metropo...",[],1565769300000
26411,Nicolás Parra,/lista/autores/nparra,Revelan identidad de 2 de 6 víctimas fatales e...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,"El intendente de Valparaíso, Jorge Martínez,...","[#derrumbe en valparaíso, #Región de Valparaís...",[],1565771100000


In [ ]:
# Obtener los valores únicos y el tamaño de cada subcategoría
valores_unicos = dataset_filtrado['subcategory'].unique()
tamanos_subcategorias = dataset_filtrado.groupby('subcategory').size()

# Crear un DataFrame con las subcategorías y sus tamaños
df_subcategorias = pd.DataFrame({
    'Subcategoría': tamanos_subcategorias.index,
    'Tamaño': tamanos_subcategorias.values
})

# Mostrar el DataFrame
df_subcategorias

,Subcategoría,Tamaño
0,actualidad-economica,1040
1,america-latina,4149
2,animales,425
3,asia,757
4,chile,1926
5,curiosidades,796
6,eeuu,2021
7,entrevistas,663
8,europa,1976
9,mediooriente,527


In [ ]:
# Determinar el número mínimo de ejemplos entre las subcategorías
min_samples = df_subcategorias['Tamaño'].min()

print(f"El número mínimo de ejemplos entre las subcategorías es: {min_samples}")

# Balancear el dataset seleccionando el número mínimo de ejemplos por subcategoría
g = dataset_filtrado.groupby('subcategory')

# Aplicamos el balanceo y nos aseguramos de que el índice se restablezca correctamente
dataset_balanceado = g.apply(lambda x: x.sample(min_samples)).reset_index(drop=True)

# Verificar el tamaño del dataset balanceado y por subcategoría
print(f"Tamaño del dataset balanceado: {len(dataset_balanceado)}")
print("Tamaño por subcategoría después del balanceo:")
print(dataset_balanceado.groupby('subcategory').size())


El número mínimo de ejemplos entre las subcategorías es: 418
Tamaño del dataset balanceado: 7942
Tamaño por subcategoría después del balanceo:
subcategory
actualidad-economica      418
america-latina            418
animales                  418
asia                      418
chile                     418
curiosidades              418
eeuu                      418
entrevistas               418
europa                    418
mediooriente              418
negocios-y-empresas       418
region-de-la-araucania    418
region-de-los-lagos       418
region-de-los-rios        418
region-de-valparaiso      418
region-del-bio-bio        418
region-metropolitana      418
tu-bolsillo               418
viral                     418
dtype: int64


/var/folders/n4/j3zj6_r13rv55gc2c5btztph0000gn/T/ipykernel_42780/3084681870.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset_balanceado = g.apply(lambda x: x.sample(min_samples)).reset_index(drop=True)


In [ ]:
dataset.sample(10)

,author,author_link,title,link,category,subcategory,content,tags,embedded_links,publication_datetime
17331,Matías Vega,/lista/autores/mvega,Sofofa responde a pedido del Gobierno de aumen...,https://www.biobiochile.cl/noticias/economia/a...,economia,actualidad-economica,La Sociedad de Fomento Fabril (Sofofa) respo...,"[#Desempleo, #Ipc, #paciencia, #Sofofa]",[https://media.biobiochile.cl/wp-content/uploa...,1533753840000
8253,Catalina Díaz,/lista/autores/catalinadiaz,Robo de 2.500 metros de cable de cobre nuevame...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-del-bio-bio,Un nuevo robo de alambre cobre del tendido e...,"[#Mulchén, #Región de la Araucanía, #Temuco]",[https://media.biobiochile.cl/wp-content/uploa...,1559147400000
7768,Manuel Cabrera,/lista/autores/mcabrera,Sigue crisis por basura en Chiloé: vecinos de ...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-los-lagos,Un complejo panorama es que se avizora para ...,"[#Basura, #Chiloé, #Crisis, #Dalcahue, #Puerto...",[https://media.biobiochile.cl/wp-content/uploa...,1558790580000
15364,Yessenia Márquez,/lista/autores/ymarquez,Inmobiliara presenta recurso de protección con...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-los-lagos,Un recurso de protección en contra del direc...,"[#Inmobiliaria Las Hortencias, #Los Angeles, #...",[https://media.biobiochile.cl/wp-content/uploa...,1562955600000
20057,Diego Vera,/lista/autores/dvera,Tras polémica con Google: EEUU da 90 días a Hu...,https://www.biobiochile.cl/noticias/internacio...,internacional,eeuu,El Departamento de Comercio de Estados Unido...,"[#Estados Unidos, #Google, #Huawei]",[],1558374600000
20248,Nicole Briones,/lista/autores/nbriones,La crisis del agua en Osorno no termina: vecin...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-los-lagos,En Osorno la crisis del agua no termina. Si ...,"[#Contaminación, #crisis del agua, #Osorno, #R...",[https://media.biobiochile.cl/wp-content/uploa...,1564312980000
20448,Emilio Lara,/lista/autores/elara,"Adelantan pago del ""Bono Marzo"": conoce los re...",https://www.biobiochile.cl/noticias/economia/t...,economia,tu-bolsillo,"La tarde de este miércoles, el ministro del ...","[#Bono, #Bono Marzo, #Economía, #Instituto de ...",[],1544631120000
1072,Diego Vera,/lista/autores/dvera,Bolsonaro se reúne con Macri en cita marcada p...,https://www.biobiochile.cl/noticias/internacio...,internacional,america-latina,"El presidente de Brasil, Jair Bolsonaro, rec...","[#Argentina, #Jair Bolsonaro, #Mauricio Macri,...",[],1547635200000
6362,Manuel Stuardo,/lista/autores/mstuardo,Juzgado de Garantía rechaza prisión preventiva...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-los-rios,El Juzgado de Garantía de Los Lagos rechazó ...,"[#ataque, #Femicidio Frustrado, #prisión preve...",[https://media.biobiochile.cl/wp-content/uploa...,1557769020000
1772,Diego Vera,/lista/autores/dvera,Guerra comercial: EEUU implementa una serie de...,https://www.biobiochile.cl/noticias/internacio...,internacional,eeuu,Estados Unidos comenzó a aplicar este lunes ...,"[#Aranceles, #China, #Estados Unidos, #Guerra ...",[],1537784100000


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset_balanceado.content,
                                                    dataset_balanceado.subcategory,
                                                    test_size=0.33,
                                                    random_state=42)

In [ ]:
X_train

724       El presidente Jair Bolsonaro reivindicó esta...
765       El Poder Electoral venezolano descartó este ...
6900      Conmoción causó en Chile y en Brasil la noti...
4317      Durante los últimos años, los volúmenes de p...
4463      La Corte Suprema desestimó las alegaciones d...
                              ...                        
5226      Un trabajo entre las fiscalías de Puerto Mon...
5390      La temperatura más fría de la temporada inve...
860       El pasado 1 de enero , mientras muchos celeb...
7603      El gimnasio suele ser el lugar donde termina...
7270      El nuevo ministro del Trabajo, Nicolás Monck...
Name: content, Length: 5321, dtype: object

In [ ]:
X_test

3286      Este miércoles se realizó una reunión en Cor...
3322      El diputado del Partido Socialista, Leonardo...
5004      Un ciudadano chileno, que fue sorprendido in...
1420      El líder norcoreano, Kim Jong Un, afirmó est...
3785      Bases militares y centros de investigación c...
                              ...                        
7552      La tranquilidad del entorno y sus aguas cris...
3570      Una polémica se registró la jornada de este ...
7395      El Gobierno postergó, una vez más, la presen...
6174      Un derrame de combustible se registró en la ...
3254      El Superintendente de Electricidad y Combust...
Name: content, Length: 2621, dtype: object

### Nuestro primer sistema de clasificación


Ahora que tenemos cargado el dataset, podemos implementar nuestro clasificador!

Para esto, usaremos 3 herramientas fundamentales de scikit-learn: un `pipeline`, `CountVectorizer` y `MultinomialNB`.

#### Pipeline 


Un [`pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) es la definición de los procesos que llevará a cabo el sistema que creemos. Nos permite tener unificados todos los procesos a la vez que simplifica el código de nuestro sistema.


En nuestro caso, el pipeline será:

    Dataset -> Bag of Words -> NaiveBayes Clf


#### Bag of Words y CountVectorizer 🎒 


¿Qué era Bag of Words?

Es un modelo en donde transformamos cada una de las oraciones de nuestro dataset en vectores. Cada vector contiene una columna por cada palabra / **token** del vocabulario. Al procesar el dataset, cada oración es mapeada a un vector que cuenta las apariciones de cada una de sus tokens. 

Referencia: [BoW en wikipedia](https://es.wikipedia.org/wiki/Modelo_bolsa_de_palabras)

**Un pequeño ejemplo**

Supongamos que nuestro tokenizador solo separa por espacios.

    - Doc1 : 'I love dogs'
    - Doc2: 'I hate dogs and knitting.
    - Doc3: 'Knitting is my hobby and my passion.

El bag of words quedaría:

`CountVectorizer` es la clase de `scikit` que transformará nuestro texto a Bag of Words. Fijense que es tremendamente útil tenerla dentro de un pipeline ya que fija en un comienzo el vocabulario que tendrá el Bag of Words, evitando discordancias entre los vectores del conjunto de entrenamiento y el de prueba.

#### MultinomialNB

El clasificador

------------------------
#### Creemos el clasificador 🧪

**Primero, definimos el pipeline**

In [ ]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer()  

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf = Pipeline([('vect', vectorizer), ('clf', clf)])

**Luego, lo entrenamos**

In [ ]:
# Entrenamos nuestro pipeline
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

**Y predecimos**

In [ ]:
y_pred = text_clf.predict(X_test)
y_pred

array(['entrevistas', 'entrevistas', 'america-latina', ..., 'chile',
       'region-de-los-lagos', 'entrevistas'], dtype='<U22')

**Veamos como nos fue:**

In [ ]:
# algunos ejemplos:
pd.DataFrame({'content': X_test, 'category':y_test, 'predicted category': y_pred}) 

,content,category,predicted category
3286,Este miércoles se realizó una reunión en Cor...,entrevistas,entrevistas
3322,"El diputado del Partido Socialista, Leonardo...",entrevistas,entrevistas
5004,"Un ciudadano chileno, que fue sorprendido in...",region-de-la-araucania,america-latina
1420,"El líder norcoreano, Kim Jong Un, afirmó est...",asia,asia
3785,Bases militares y centros de investigación c...,mediooriente,mediooriente
...,...,...,...
7552,La tranquilidad del entorno y sus aguas cris...,viral,curiosidades
3570,Una polémica se registró la jornada de este ...,europa,europa
7395,"El Gobierno postergó, una vez más, la presen...",tu-bolsillo,chile
6174,Un derrame de combustible se registró en la ...,region-de-valparaiso,region-de-los-lagos


In [ ]:
# usando la matriz de confusión:

# eje x -> predichos
# eje y -> clase real

print(confusion_matrix(y_test, y_pred))

[[ 92   1   0   0  20   2   1   1   1   0   6   0   0   0   1   2   0  21
    0]
 [  0 129   0   1   4   5   4   0   3   1   0   0   0   0   0   0   1   0
    0]
 [  0   2 106   1   0  12   1   0   2   1   0   0   0   0   0   0   0   0
   15]
 [  1   1   7 121   0   6  10   0   4   3   0   0   0   0   0   0   0   0
    0]
 [  5   1   0   0  91   2   0   4   0   0   0   0   1   0   4   5   7   9
    0]
 [  0   0   3   5   0 104   3   0   3   1   1   0   0   0   0   0   0   4
   12]
 [  0   9   1   6   0  12  98   0   2  10   0   0   0   0   0   0   0   0
    0]
 [  1   1   0   0   3   3   0 124   2   1   0   0   0   0   0   0   2   2
    0]
 [  0   3   1   2   0   5   3   0  97   6   0   0   0   0   0   0   0   1
    0]
 [  0   1   0   2   0   3   2   0   4 130   0   0   0   0   0   0   0   0
    0]
 [ 28   2   0   1  20   6   2   1   1   0  52   0   1   0   1   3   0  36
    0]
 [  1   1   0   0  29   0   0   2   2   0   2  63   5   2  11   3  22   3
    1]
 [  2   0   0   0  17   1   

#### Métricas de Evaluación

Las métricas definen un puntaje de evaluación que indica que tal le fue al sistema. Hay muchas formas distintas de medir su rendimiento. Entre estas, tenemos:

- `precision`: El número de documentos de una clase clasificados correctamente dividido por el número de documentos totales clasificados como esa clase.

- `recall`: El número de documentos de una clase clasificados correctamente dividido por el número de los documentos que se deberían haber clasificado como esa clase.(número de documentos reales de esa clase).

- `f1-score` : Es la media armónica entre los anteriores.

- `accuracy` : La cantidad de documentos clasificados correctamente versus todos los documentos

Por otra parte, tenemos dos formas de ver dichas métricas agrupadas:

- `Macroaveraging`:    Se computan las métricas por cada clase y luego de promedia.

- `Microaveraging`:    Se recolectan las clasificaciones por cada clase, se computa la tabla de contingencia (todos los elementos clasificados) y se evalua. Representa un Macroaveraging ponderado por el número de miembros de una clase.

In [ ]:
# usando el classification report:
print(classification_report(y_test, y_pred))

                        precision    recall  f1-score   support

  actualidad-economica       0.63      0.62      0.63       148
        america-latina       0.85      0.87      0.86       148
              animales       0.83      0.76      0.79       140
                  asia       0.85      0.79      0.82       153
                 chile       0.32      0.71      0.44       129
          curiosidades       0.48      0.76      0.59       136
                  eeuu       0.77      0.71      0.74       138
           entrevistas       0.91      0.89      0.90       139
                europa       0.80      0.82      0.81       118
          mediooriente       0.85      0.92      0.88       142
   negocios-y-empresas       0.81      0.34      0.48       154
region-de-la-araucania       0.97      0.43      0.59       147
   region-de-los-lagos       0.79      0.66      0.72       138
    region-de-los-rios       0.93      0.53      0.68       141
  region-de-valparaiso       0.69      

#### Ejecutemos algunas consultas!

In [ ]:
text_clf.predict([
    ("En puerto montt se encontró un perrito, que aparentemente,"
    "habría consumido drogas de alto calibre. Producto de esto,"
    "se ponostíca que padecerá severa caña durante varios dias.")
])

array(['region-de-los-lagos'], dtype='<U22')

In [ ]:
text_clf.predict(["kim jong un será el próximo candidato a ministro de educación."])

array(['asia'], dtype='<U22')

In [ ]:
text_clf.predict([("El banco mundial presentó para chile un decrecimiento"
                   "económico de 92% y una inflación de 8239832983289%.")])

array(['actualidad-economica'], dtype='<U22')

-------------------------

Se ven bastante buenos los resultados. ¿Pero, podremos mejorarlos?

### Preprocesamiento del texto

Podemos preprocesar los textos?, para intentar mejorar.
Es decir, cómo hacemos el proceso de tokenización (separación de las palabras).



Alguna de las técnicas son:


- Eliminación de Stopwords
- Stemming
- Lematización

Existen otros preprocesadores que agregan información a las oraciones, tales como aquellos que indican negaciones.

A continuación, describiremos con mas detalle cada uno de estas técnicas.

#### Tokenizar ➗

¿Qué era tokenizar?


    Es el proceso de convertir una secuencia de carácteres (por ejemplo, una oración) en una secuencia de valores distintos entre si llamados tokens.
    
Referencia: [Tokenización en wikipedia](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)



**spaCy y el objeto nlp**

`nlp` es el objeto que nos permite usar e interactuar con la librería [`spacy`](https://spacy.io/).
Esta librería incluye variadas herramientras, tales como tokenizar, lematizar, descartar stopwords, entre otras (para este auxiliar, solo utilizaremos las mencionadas). El objeto nlp lo instanciamos en la sección de imports.

Para usarla, simplemente se le pasa el texto como parámetro, como veremos en el siguiente ejemplo: 

In [ ]:
DOC = "hermanito mio te estas pegando el show"

tokens = []
for word in nlp(DOC):
    tokens.append(word)

tokens

[hermanito, mio, te, estas, pegando, el, show]

**Observación**: Para este auxiliar usaremos `List Comprehension`, otra forma de hacer un for un poco mas reducida.
Una muy buena referencia de esto [aquí](https://www.programiz.com/python-programming/list-comprehension).

La operación anterior usando esta sintáxis quedaría como:

In [ ]:
tokens = [word for word in nlp(DOC)]
tokens

[hermanito, mio, te, estas, pegando, el, show]

#### Stopwords 🛑

¿Qué eran las stopwords?

    Las Stopwords son palabras muy comunes en nuestro lenguaje y que por lo tanto, no aportan mucha información. Existen múltiples listas de stopwords para muchos idiomas y la aplicación de estas variará caso a caso.

    
Referencias: [Stopwords en Wikipedia](https://en.wikipedia.org/wiki/Stop_words)

En este caso, utilizaremos las stopwords inlcuidas en la librería spaCy en español

In [ ]:
print(len(STOP_WORDS))
print(list(STOP_WORDS)[0:20])

521
['seis', 'vuestra', 'algún', 'sus', 'debajo', 'saber', 'mías', 'lado', 'o', 'tenido', 'antes', 'da', 'temprano', 'podría', 'podriais', 'suyo', 'través', 'ti', 'acuerdo', 'cuál']


#### Stemming 🔪

¿Qué era el stemming? 

    Son un conjunto de métodos enfocados en reducir cada palabra a su raiz.

Referencia: [Stemming en Wikipedia](https://en.wikipedia.org/wiki/Stemming)
  
**Ejemplos: **


| word | stem of the word  |
|---|---|
working | work
worked | work
works | work

**nltk**

En este caso, utilizaremos la segunda librería de herramientas de nlp: [`nltk`](https://www.nltk.org/). Esta provee una buena herramienta para hacer stemming en español : `SnowballStemmer`

In [ ]:
stemmer = SnowballStemmer('spanish')
stemmed_doc = [stemmer.stem(str(token)) for token in tokens]
print(stemmed_doc)

['hermanit', 'mio', 'te', 'estas', 'peg', 'el', 'show']


#### Lematización 🙀

¿Qué era lematización? 

    Es el proceso de transformar cada token a su lema, el cual es la palabra base sin ningún tipo de flexión o alteración como las conjugaciones, por ejemplo.
    
    

  
    
    
Referencia: [Lematización en wikipedia](https://en.wikipedia.org/wiki/Lemmatisation)

Refernecia: [Flexión en las palabras](https://es.wikipedia.org/wiki/Flexi%C3%B3n_(ling%C3%BC%C3%ADstica))

**Ejemplos**

| word | lemma  |
|---|---|
dije| decir 
guapas | guapo
mesa | mesas


 <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Flexi%C3%B3nGato-svg.svg/300px-Flexi%C3%B3nGato-svg.svg.png" alt="Flexión de gato" style="width: 200px;"/>
 

**Lematizar el texto**

Al igual que la tokenización, utilizaremos `scpaCy` (a través del objeto `nlp`) para lematizar el contenido.

In [ ]:
lemmatized_content = [word.lemma_ for word in nlp(DOC)]
print(lemmatized_content)

['hermanito', 'mio', 'tú', 'este', 'pegar', 'el', 'show']


**Discusión:**

    ¿Cuál es mejor?

### Sistema de clasificación con preprocesamiento

Para agregar los tokenizadores en el sistema, creamos funciones que que cada documento de forma individual usando nuestro preprocesador favorito. Luego, `CountVectorizer` se encargará de usar estas funciones sobre todo el dataset.

In [ ]:
# Tokenizers para CountVectorizer

# Solo tokenizar el doc usando spacy.
def tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]


# Tokenizar y remover las stopwords del doc
def tokenizer_with_stopwords(doc):
    return [x.orth_ for x in nlp(doc) if x.orth_ not in STOP_WORDS]


# Tokenizar y lematizar.
def tokenizer_with_lemmatization(doc):
    return [x.lemma_ for x in nlp(doc)]

# Tokenizar y hacer stemming.
def tokenizer_with_stemming(doc):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in [x.orth_ for x in nlp(doc)]]

#### Creamos nuestro clasificador


**Definimos el pipeline**

In [ ]:
# seleccionamos el tokenizador a usar:
TOKENIZER = tokenizer_with_stemming

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 1))

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf_2 = Pipeline([('vect', vectorizer), ('clf', clf)])

**Entrenamos nuestro pipeline y predecimos**


In [ ]:
text_clf_2.fit(X_train, y_train)
y_pred = text_clf_2.predict(X_test)

/Users/pavt/Documents/personal/ufro/2024-2/Ing-datos-civil/id-civil/.id_civil/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[ 88   1   0   0  21   2   2   1   1   0   6   0   0   0   1   2   0  23
    0]
 [  1 130   0   1   3   5   4   0   3   0   0   0   0   0   0   0   1   0
    0]
 [  0   2 110   0   0  10   2   0   2   0   0   0   0   0   0   0   0   0
   14]
 [  2   0   8 119   0   6  11   0   4   3   0   0   0   0   0   0   0   0
    0]
 [  5   2   0   0  91   1   0   5   0   0   0   1   1   0   4   5   6   8
    0]
 [  0   0   3   4   0 107   5   0   1   1   0   0   0   0   0   0   1   4
   10]
 [  0  10   2   7   0  10  94   0   5  10   0   0   0   0   0   0   0   0
    0]
 [  2   2   0   0   6   5   0 118   1   1   0   0   0   0   0   1   2   1
    0]
 [  0   3   1   1   0   5   3   0 100   4   0   0   0   0   0   0   0   1
    0]
 [  0   1   0   2   0   3   2   0   4 130   0   0   0   0   0   0   0   0
    0]
 [ 25   2   0   1  22   6   2   1   1   0  54   0   1   0   1   3   0  35
    0]
 [  1   0   0   0  31   1   0   1   0   0   2  70   4   1  11   3  20   2
    0]
 [  1   0   0   0  15   1   

#### Pregunta abierta: ¿Por qué no mejoran los resultados?

[Aquí]("https://www.quora.com/Is-it-normal-to-get-better-accuracy-without-stemming-and-lemmatization-than-using-them-in-NLP-text-classification") hay una muy buena discusión al respecto.

### Clasificación usando Regresión Logísitica

No profundizaremos en este clasificador, mas del hecho de que se "supone" que debería tener mejor rendimiento que el de bayes.

Referencia: [Regresión Logística](https://en.wikipedia.org/wiki/Logistic_regression)

**Definimos nuestro Pipeline**

In [ ]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 1))

# Ahora definimos regresión logística como clasificador.
log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

**Entrenamos y predecimos**

In [ ]:
log_pipe.fit(X_train, y_train)
y_pred = log_pipe.predict(X_test)

/Users/pavt/Documents/personal/ufro/2024-2/Ing-datos-civil/id-civil/.id_civil/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/pavt/Documents/personal/ufro/2024-2/Ing-datos-civil/id-civil/.id_civil/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


**Evaluamos**

In [ ]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[ 87   1   0   0  12   3   2   0   1   0  17   0   0   0   1   7   0  15
    2]
 [  0 118   0   1   5   2   7   0   9   0   0   1   0   0   0   0   0   2
    3]
 [  0   1 113   3   0   6   1   0   1   0   0   0   0   1   0   0   0   0
   14]
 [  0   3   9 113   0   5  12   0   4   5   1   0   0   0   0   0   0   0
    1]
 [  6   0   0   1  77   2   1   2   0   0   7   5   1   0   5   5  12   5
    0]
 [  0   0   7   6   1  78   3   0   5   1   2   0   0   0   1   0   0   5
   27]
 [  0  10   1   8   1   7  93   0   5   8   0   0   0   0   1   1   0   0
    3]
 [  1   0   0   0   0   1   0 129   1   1   0   1   0   0   0   0   5   0
    0]
 [  0   3   2   0   1   4   4   0  97   5   0   0   0   0   0   0   1   0
    1]
 [  1   0   0   5   0   4   5   0   4 119   1   0   0   0   0   0   1   1
    1]
 [ 17   3   0   0   3   5   0   1   1   0  92   0   1   1   2   6   5  16
    1]
 [  0   0   0   0   4   0   0   1   0   0   1 128   1   2   2   4   2   1
    1]
 [  0   0   0   0   1   0   

### N-gramas

Los n-gramas son conjuntos de n-tokens seguidos entre si. La idea de usar esto es que además, capturemos conceptos. 

Por ejemplo, si usamos 2-gramas sobre `'Hoy día comí lentejas'`, esta quedaría como:


```python
['hoy dia', 'día comí', 'comí lentejas']
```

`CountVectorizer` tiene la opción para poner n-gramas del tamaño que tu quieras, y además incluir mas pequeños. Todo esto se define en el parámetro `ngram_range`. Este recibe una tupla con los rangos del n-grama mas pequeño y el mas grande. Por ejemplo, para (1,2), la oración anterior quedaría: 


```python
['hoy', 'día', 'comí', 'lentejas', 'hoy dia', 'día comí', 'comí lentejas']
```

Nota que esto incrementa el tamaño de los vectores de Bag of words y por lo tanto, del entrenamiento y de la predicción. 

In [ ]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                     
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 3))

# Ahora definimos regresión logística como clasificador.
log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

**Entrenamos y predecimos**

In [ ]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[ 87   1   0   0  12   3   2   0   1   0  17   0   0   0   1   7   0  15
    2]
 [  0 118   0   1   5   2   7   0   9   0   0   1   0   0   0   0   0   2
    3]
 [  0   1 113   3   0   6   1   0   1   0   0   0   0   1   0   0   0   0
   14]
 [  0   3   9 113   0   5  12   0   4   5   1   0   0   0   0   0   0   0
    1]
 [  6   0   0   1  77   2   1   2   0   0   7   5   1   0   5   5  12   5
    0]
 [  0   0   7   6   1  78   3   0   5   1   2   0   0   0   1   0   0   5
   27]
 [  0  10   1   8   1   7  93   0   5   8   0   0   0   0   1   1   0   0
    3]
 [  1   0   0   0   0   1   0 129   1   1   0   1   0   0   0   0   5   0
    0]
 [  0   3   2   0   1   4   4   0  97   5   0   0   0   0   0   0   1   0
    1]
 [  1   0   0   5   0   4   5   0   4 119   1   0   0   0   0   0   1   1
    1]
 [ 17   3   0   0   3   5   0   1   1   0  92   0   1   1   2   6   5  16
    1]
 [  0   0   0   0   4   0   0   1   0   0   1 128   1   2   2   4   2   1
    1]
 [  0   0   0   0   1   0   

### Bonus: Clasificación de Autoría de documentos

¿Existirá un patrón en como escriben los periodistas que nos permitan identificarlos a partir de sus textos?

In [ ]:
dataset_r.author.value_counts()[0:20]

author
Diego Vera               4471
Emilio Lara              1380
Matías Vega              1230
César Vega Martínez      1202
María José Villarroel    1157
Gonzalo Cifuentes        1122
Manuel Stuardo           1022
Manuel Cabrera            999
Valentina González        966
Nicole Briones            850
Felipe Delgado            825
Yessenia Márquez          764
Paola Alemán              760
Verónica Reyes            718
Jonathan Flores           618
Nicolás Díaz              592
Sebastián Asencio         544
Catalina Díaz             523
Ariela Muñoz              515
Nicolás Parra             503
Name: count, dtype: int64

In [ ]:
NUM_SAMPLES = 250

def process_datasets_by_author(dataset):
    
    # creamos una nueva columna titulo y contenido.
    content = dataset['title'] + '. ' + dataset['content'] 
    # obtenemos las clases
    subcategory = dataset.author
    # dejamos en el dataset solo contenido de la noticia y categoria
    dataset = pd.DataFrame({'content': content, 'author': subcategory})

    selected_authors = ['Diego Vera', 'Emilio Lara', 'Matías Vega', 'César Vega Martínez',
           'María José Villarroel', 'Gonzalo Cifuentes', 'Manuel Stuardo',
           'Manuel Cabrera', 'Valentina González', 'Nicole Briones',
           'Felipe Delgado', 'Yessenia Márquez', 'Paola Alemán', 'Verónica Reyes',
           'Jonathan Flores', 'Nicolás Díaz', 'Sebastián Asencio', 'Catalina Díaz',
           'Ariela Muñoz', 'Nicolás Parra']

    # filtrar solo categorias seleccionadas
    dataset = dataset[dataset['author'].isin(selected_authors)]

    # balancear clases
    g = dataset.groupby('author')
    dataset = pd.DataFrame(
        g.apply(lambda x: x.sample(NUM_SAMPLES).reset_index(drop=True))).reset_index(
            drop=True)
    
    return dataset

    



In [ ]:
author_dataset = process_datasets_by_author(dataset_r.copy(deep=True))

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    author_dataset.content,
    author_dataset.author,
    test_size=0.33,
    random_state=42)

/var/folders/n4/j3zj6_r13rv55gc2c5btztph0000gn/T/ipykernel_42780/4182984880.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  g.apply(lambda x: x.sample(NUM_SAMPLES).reset_index(drop=True))).reset_index(


In [ ]:
author_dataset

,content,author
0,"Salarios crecen 1,2% pese al alza de la fuerza...",Ariela Muñoz
1,Ecuador aprueba matrimonio igualitario en hist...,Ariela Muñoz
2,Los 10 compromisos que firmó el Gobierno para ...,Ariela Muñoz
3,Gerente de CCU ante eventual alza de impuesto ...,Ariela Muñoz
4,Erupción de volcán al suroeste de Japón elevó ...,Ariela Muñoz
...,...,...
4995,"""Traidores nunca, leales siempre"": arenga Madu...",Yessenia Márquez
4996,Solicitan instalación de dispensadores de agua...,Yessenia Márquez
4997,Detienen a hombre en Atacama que viajaba con 9...,Yessenia Márquez
4998,Inician sumario sanitario en colegio Darío Sal...,Yessenia Márquez


#### Definir el Pipeline

In [ ]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range=(1, 1))

# Definimos el clasificador. Usaremos bayes, ya que regresión logística se demora 1/4 del tiempo del universo.
clf = MultinomialNB()   


# Creamos el pipeline
log_pipe_by_author = Pipeline([('vect', vectorizer), ('clf', clf)])

#### Entrenar

In [ ]:
log_pipe_by_author.fit(X_train_2, y_train_2)
y_pred = log_pipe_by_author.predict(X_test_2)

#### Evaluar

In [ ]:
# usando la matriz de confusión:
print(confusion_matrix(y_test_2, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test_2, y_pred))

[[ 4  5  1 14  0  4  1 27  0  2  7  1  1 10  0  1  0  3 10  3]
 [ 1 32  1  1  1  0  0  2  2  8  4  0  6 11  1  2  2  2  1  8]
 [ 0  0 57  7  0  1  0  2  0  0  0  0  0  0  0 23  0  0  0  0]
 [ 0  0  1 65  0  0  0  0  0  0  0  0  0  0  0  6  0  0  0  0]
 [ 1  0  0 34  1  2  0 27  0  0  2  0  0  4  1  6  0  2 14  2]
 [ 0  1  0 19  0 33  0  9  0  0  6  0  0  0  0  5  0  2  0  2]
 [ 1  0  2 29  0  7  2 26  0  0  2  1  0  4  0 11  0  1  6  0]
 [ 1  0  0 15  0  0  0 55  0  0  0  0  0  1  0  4  0  0  1  0]
 [ 1  2  1 13  0  3  0  9  7  0  1  1  0 18 12  7  1  3  0  3]
 [ 1  2  0  8  0  1  0 10  1  9  6  1  8 16  0  4  0  1  1 16]
 [ 0  1  0  6  0  8  0  9  0  1 26  0  0  9  1  9  0  2  0  5]
 [ 0  0  0 26  0  6  0 31  1  1  3  3  1  6  1  2  0  0  2  0]
 [ 1  9  0  5  0  1  0  7  0 12  0  0 15 23  2  3  0  1  1  7]
 [ 1  1  0  2  0  5  0 15  0  1  3  0  1 37  2  0  0  1  1  3]
 [ 0  3  1 14  0  2  0 12  4  1  3  0  0 19 13  4  2  1  1  0]
 [ 0  0  1 38  0  0  0  1  0  0  0  0  0  0  0 45  0  0

## Referencias
 

Slides:
- https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf


Análisis de sentimientos como clasificación de texto:
- https://affectivetweets.cms.waikato.ac.nz/benchmark/

Algunos Recursos útiles
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [Scikit-learn Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf)
- [Spacy Tutorial](https://www.datacamp.com/community/blog/spacy-cheatsheet)
- [NLTK Cheat sheet](http://sapir.psych.wisc.edu/programming_for_psychologists/cheat_sheets/Text-Analysis-with-NLTK-Cheatsheet.pdf)